# Multi-Agent AI Classification System


### This notebook demonstrates a hands-on implementation of a multi-agent AI system using generative AI models. The goal is to classify input data using agent-based architecture rather than rule-based logic.

Technologies used:
- Python
- Generative AI (Gemini)
- Multi-agent architecture
- Prompt-based classification
- Kaggle Notebook environment


In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audi-car-price-and-specifications-dataset/cars_dataset.csv


In [69]:
!pip install google-adk --no-deps -q
!pip install pdfplumber -q


In [70]:
import os
from kaggle_secrets import UserSecretsClient

# Load API key from Kaggle Secrets
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

# Set it as an environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("API Key Loaded:", "Yes ✅" if GOOGLE_API_KEY else "No ❌")


API Key Loaded: Yes ✅


In [71]:
from google.adk.agents import Agent
import os
import pandas as pd
import pdfplumber

class IngestAgent(Agent):
    # Required by ADK / Pydantic
    name: str = "ingest_agent"
    description: str = "Reads text or local files and returns plain text."

    def run(self, input_data: str) -> str:
        """Ingest raw text or a local file path and return plain text."""

        # If input is a file path, read file
        if os.path.isfile(input_data):
            ext = input_data.split(".")[-1].lower()

            if ext == "txt":
                with open(input_data, "r", encoding="utf-8") as f:
                    return f.read()

            elif ext == "csv":
                df = pd.read_csv(input_data)
                # Join all rows into one big text
                return "\n".join(df.astype(str).apply(" ".join, axis=1))

            elif ext == "pdf":
                text = ""
                with pdfplumber.open(input_data) as pdf:
                    for page in pdf.pages:
                        page_text = page.extract_text() or ""
                        text += page_text + "\n"
                return text

        # Otherwise, treat it as direct text
        return input_data


In [72]:
from google.adk.agents import Agent


In [73]:
from google.adk.agents import Agent
import google.generativeai as genai
from google.api_core.exceptions import NotFound
import os

# Configure Gemini from env API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Simple local rule-based backup (no internet needed)
def rule_based_category(text: str) -> str:
    t = text.lower()
    if any(w in t for w in ["revenue", "profit", "market", "business", "startup", "company"]):
        return "Business"
    if any(w in t for w in ["ai", "machine learning", "deep learning", "python", "software", "technology", "data"]):
        return "Technology"
    if any(w in t for w in ["success", "inspiration", "motivation", "dream", "goal", "mindset"]):
        return "Motivation"
    return "Other"


class ClassifierAgent(Agent):
    name: str = "classifier_agent"
    description: str = "Classifies text using Gemini; falls back to rule-based if Gemini is not available."

    def run(self, text: str) -> str:
        # 1️⃣ Try Gemini
        try:
            # Try a couple of common model names
            for model_name in [
                "gemini-1.5-flash",
                "gemini-1.5-flash-latest",
                "gemini-1.0-pro",
                "gemini-pro",
            ]:
                try:
                    model = genai.GenerativeModel(model_name)
                    prompt = f"""
                    You are a text classifier.

                    Classify the following text into exactly ONE of these categories:
                    [Business, Technology, Motivation, Other]

                    Text:
                    {text}

                    Answer with ONLY the category name.
                    """
                    response = model.generate_content(prompt)
                    category = (response.text or "").strip()
                    if category:
                        print(f"Using Gemini model: {model_name}")
                        return category
                except NotFound:
                    # Try next model name
                    continue

            print("⚠️ No Gemini model name worked for this API key. Using rule-based classifier.")
            return rule_based_category(text)

        # 2️⃣ Network / other errors → fallback
        except Exception as e:
            print("⚠️ Gemini call failed, falling back to rule-based classifier.")
            print("Error:", e)
            return rule_based_category(text)


In [74]:
ingest = IngestAgent()
classifier = ClassifierAgent()

sample_input = "Artificial intelligence will transform future business models."
text = ingest.run(sample_input)
category = classifier.run(text)

print("Input Text:", text)
print("📌 Classified Category:", category)


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
Input Text: Artificial intelligence will transform future business models.
📌 Classified Category: Business


In [75]:
# Example: classify a PDF file
file_path = "/kaggle/input/your_dataset/sample.pdf"  # <-- change this

text_from_file = ingest.run(file_path)
category_from_file = classifier.run(text_from_file)

print("First 500 chars of file text:\n", text_from_file[:500])
print("\n📌 File Classified Category:", category_from_file)


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
First 500 chars of file text:
 /kaggle/input/your_dataset/sample.pdf

📌 File Classified Category: Technology


In [76]:
import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/audi-car-price-and-specifications-dataset/cars_dataset.csv


In [77]:
import os
import pandas as pd

# ✅ Auto-find your uploaded file path
file_path = None
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.lower().endswith(".csv"):  # finds first CSV
            file_path = os.path.join(root, file)
            break

file_path  # display found path


'/kaggle/input/audi-car-price-and-specifications-dataset/cars_dataset.csv'

In [78]:
import os

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.lower().endswith(".pdf"):
            print(os.path.join(root, file))


In [79]:
pdf_path = "/kaggle/input/👉PASTE_YOUR_COPIED_PATH_HERE.pdf"

# Extract text using your ingest agent
text = ingest.run(pdf_path)

# Classify the extracted text
category = classifier.run(text)

print("📄 PDF text preview:\n", text[:400])
print("🧠 Classified Category:", category)


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
📄 PDF text preview:
 /kaggle/input/👉PASTE_YOUR_COPIED_PATH_HERE.pdf
🧠 Classified Category: Other


In [80]:
import pandas as pd
import os

# ✅ Your Kaggle CSV path (auto-detect)
file_path = None
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.lower().endswith(".csv"):
            file_path = os.path.join(root, file)
            break

df = pd.read_csv(file_path)
print("✅ CSV Loaded from:", file_path)
df.head()


✅ CSV Loaded from: /kaggle/input/audi-car-price-and-specifications-dataset/cars_dataset.csv


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,Make
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi


In [81]:
# Build a human-readable text description for each car
text_columns = ["Make", "model", "transmission", "fuelType"]

df["car_text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

# Just to check:
df[["car_text"]].head()


,car_text
0,audi A1 Manual Petrol
1,audi A6 Automatic Diesel
2,audi A1 Manual Petrol
3,audi A4 Automatic Diesel
4,audi A3 Manual Petrol


In [82]:
# ⚡ Fast classifier that skips Gemini completely
def fast_classifier(text: str) -> str:
    # uses your existing rule_based_category function
    return rule_based_category(text)


In [83]:
import pandas as pd

# 1️⃣ Build a text column from your car features
text_cols = ["Make", "model", "transmission", "fuelType"]
df["car_text"] = df[text_cols].astype(str).agg(" ".join, axis=1)

# 2️⃣ Simple rule-based category for cars (you can change logic later)
def rule_based_category(text: str) -> str:
    t = text.lower()
    if "diesel" in t:
        return "Diesel car"
    if "petrol" in t:
        return "Petrol car"
    if "hybrid" in t or "electric" in t:
        return "Eco / Hybrid car"
    return "Other car"

# 3️⃣ Fast classifier that does NOT call Gemini (local only)
def fast_classifier(text: str) -> str:
    return rule_based_category(text)

# 4️⃣ Apply to the new text column
TEXT_COL = "car_text"   # our combined description column

df["predicted_category"] = df[TEXT_COL].astype(str).apply(fast_classifier)

# 5️⃣ Show a few results
df[[TEXT_COL, "predicted_category"]].head()


,car_text,predicted_category
0,audi A1 Manual Petrol,Petrol car
1,audi A6 Automatic Diesel,Diesel car
2,audi A1 Manual Petrol,Petrol car
3,audi A4 Automatic Diesel,Diesel car
4,audi A3 Manual Petrol,Petrol car


In [84]:
def fast_classifier(text: str) -> str:
    return rule_based_category(text)

df["predicted_category"] = df["car_text"].astype(str).apply(fast_classifier)


In [85]:
print("IngestAgent name:", ingest.name)
print("IngestAgent description:", ingest.description)

print("ClassifierAgent name:", classifier.name)
print("ClassifierAgent description:", classifier.description)


IngestAgent name: ingest_agent
IngestAgent description: Reads text or local files and returns plain text.
ClassifierAgent name: classifier_agent
ClassifierAgent description: Classifies text using Gemini; falls back to rule-based if Gemini is not available.


In [86]:
ingest = IngestAgent()


In [87]:
# ✅ Correct variable names
print("IngestAgent name:", ingest.name)
print("IngestAgent description:", ingest.description)

print("ClassifierAgent name:", classifier.name)
print("ClassifierAgent description:", classifier.description)


IngestAgent name: ingest_agent
IngestAgent description: Reads text or local files and returns plain text.
ClassifierAgent name: classifier_agent
ClassifierAgent description: Classifies text using Gemini; falls back to rule-based if Gemini is not available.


In [88]:
demo_input = "Audi A6 Automatic Diesel"
print("Raw text:", demo_input)

ingested_text = ingest.run(demo_input)
print("\nAfter :contentReference[oaicite:0]{index=0}:", ingested_text)

classified_text = classifier.run(ingested_text)
print("After :contentReference[oaicite:1]{index=1} + :contentReference[oaicite:2]{index=2}:", classified_text)


Raw text: Audi A6 Automatic Diesel

After :contentReference[oaicite:0]{index=0}: Audi A6 Automatic Diesel
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
After :contentReference[oaicite:1]{index=1} + :contentReference[oaicite:2]{index=2}: Diesel car


In [89]:
def classify_free_text(text: str) -> str:
    """Use IngestAgent + ClassifierAgent on any text you pass."""
    ingested = ingest.run(text)
    category = classifier.run(ingested)
    return category


In [90]:
user_input = "Tesla is building self-driving electric cars."
print("Text:", user_input)
print("Category:", classify_free_text(user_input))


Text: Tesla is building self-driving electric cars.
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
Category: Eco / Hybrid car


In [91]:
def classify_car(make, model, transmission, fuel_type):
    text = f"{make} {model} {transmission} {fuel_type}"
    ingested = ingest.run(text)
    category = classifier.run(ingested)   # or fast_classifier(text) if you only want rules
    return text, category


In [92]:
text, cat = classify_car(
    make="BMW",
    model="3 Series",
    transmission="Automatic",
    fuel_type="Diesel"
)

print("Car text:", text)
print("Predicted category:", cat)


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
Car text: BMW 3 Series Automatic Diesel
Predicted category: Diesel car


In [93]:
car_text, cat = classify_car(
    make="BMW",
    model="3 Series",
    transmission="Automatic",
    fuel_type="Diesel"
)

print("Car description:", car_text)
print("Agent category:", cat)


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
Car description: BMW 3 Series Automatic Diesel
Agent category: Diesel car


In [94]:
TEXT_COL = "car_text"      # column with combined text
df["predicted_category"] = df[TEXT_COL].astype(str).apply(fast_classifier)

df[[TEXT_COL, "predicted_category"]].head()


,car_text,predicted_category
0,audi A1 Manual Petrol,Petrol car
1,audi A6 Automatic Diesel,Diesel car
2,audi A1 Manual Petrol,Petrol car
3,audi A4 Automatic Diesel,Diesel car
4,audi A3 Manual Petrol,Petrol car


In [95]:
user_text = input("Type anything about a car or topic: ")
print("You:", user_text)
print("Agent:", classify_free_text(user_text))


Type anything about a car or topic:  Mercedes SUV hybrid makes me feel proud and future-ready


You: Mercedes SUV hybrid makes me feel proud and future-ready
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.
Agent: Eco / Hybrid car


In [96]:
def run_text_demo():
    text = input("\nPaste or type your text to classify (type 'exit' to stop): ")
    if text.lower() == "exit":
        return False

    # Agent 1: Ingest
    ingested = ingest.run(text)
    print("\n🤖 [IngestAgent] Processed Text Preview:")
    print(ingested[:400] + "…" if len(ingested) > 400 else ingested)

    # Agent 2: Classify
    cat = classifier.run(ingested)
    print("\n🧠 [ClassifierAgent] Predicted Category:")
    print(cat)

    return True

# Loop for communication
while True:
    if not run_text_demo():
        print("✅ Demo finished.")
        break



Paste or type your text to classify (type 'exit' to stop):  I want a diesel car but I fear maintenance cost



🤖 [IngestAgent] Processed Text Preview:
I want a diesel car but I fear maintenance cost
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.

🧠 [ClassifierAgent] Predicted Category:
Diesel car



Paste or type your text to classify (type 'exit' to stop):  stop



🤖 [IngestAgent] Processed Text Preview:
stop
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.

🧠 [ClassifierAgent] Predicted Category:
Other car



Paste or type your text to classify (type 'exit' to stop):  exit


✅ Demo finished.


In [97]:
def run_car_demo():
    print("\n=========== 🚗 CAR INPUT DEMO ===========")
    make = input("Enter Car Make: ")
    model = input("Enter Car Model: ")
    transmission = input("Transmission (Manual/Automatic): ")
    fuel = input("Fuel Type (Petrol/Diesel/Hybrid/Electric): ")

    combined = f"{make} {model} {transmission} {fuel}"
    print("\nText sent to agents:", combined)

    # Pass through real ADK agents
    ingested = ingest.run(combined)
    car_category = classifier.run(ingested)

    print("\n🤖 [IngestAgent] Output:", ingested)
    print("🧠 [ClassifierAgent] Output:", car_category)


In [98]:
run_car_demo()



=========== 🚗 CAR INPUT DEMO ===========


Enter Car Make:  audi
Enter Car Model:  s4
Transmission (Manual/Automatic):  Manual
Fuel Type (Petrol/Diesel/Hybrid/Electric):  Petrol



Text sent to agents: audi s4 Manual Petrol
⚠️ No Gemini model name worked for this API key. Using rule-based classifier.

🤖 [IngestAgent] Output: audi s4 Manual Petrol
🧠 [ClassifierAgent] Output: Petrol car


In [99]:
print("""
=========== 🧠 Multi-Agent Capstone Architecture ===========
Agent 1: IngestAgent  → Reads/Cleans text from input or files
Agent 2: ClassifierAgent → Tries Gemini model using GOOGLE_API_KEY, else falls back offline
Batch engine uses: rule_based_category() for large CSV practical performance
=========================================================
""")

print("IngestAgent ADK object:", ingest)
print("Classifier ADK object:", classifier)



=========== 🧠 Multi-Agent Capstone Architecture ===========
Agent 1: IngestAgent  → Reads/Cleans text from input or files
Agent 2: ClassifierAgent → Tries Gemini model using GOOGLE_API_KEY, else falls back offline
Batch engine uses: rule_based_category() for large CSV practical performance

IngestAgent ADK object: name='ingest_agent' description='Reads text or local files and returns plain text.' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='' instruction='' global_instruction='' static_instruction=None tools=[] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None on_tool_error_callback=None
Classifier ADK object: name='classifier_agent' description='Classifies text using Gemini; falls

In [100]:
classify_free_text("This BMW 3 Series has an automatic diesel engine.")


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.


'Diesel car'

In [101]:
classify_free_text("I am planning to buy a small petrol hatchback for city driving.")


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.


'Petrol car'

In [102]:
classify_free_text("I went to the mall yesterday and watched a movie.")


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.


'Other car'

In [103]:
user_input = input("\nAsk something (text or car description): ")

output = classifier.run(user_input)   # ✅ fixed object name

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Electric cars inspire hope but charging worries me


⚠️ No Gemini model name worked for this API key. Using rule-based classifier.

🤖 Agent Response:

Eco / Hybrid car


In [104]:
from google.adk.agents import Agent
import google.generativeai as genai
import os

# Configure Gemini (future ready)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

class ReasoningEmotionAgent(Agent):
    name: str = "reasoning_emotion_agent"
    description: str = "Understands emotion + performs reasoning and classifies cars or topics."

    def run(self, text: str) -> str:
        model = genai.GenerativeModel("gemini-1.5-flash")  # future model call

        prompt = f"""
        You are an intelligent assistant with emotional understanding and reasoning abilities.

        1. Detect the emotion/sentiment of the user text.
        2. Reason about the text meaning and intention.
        3. If the text describes a car, classify it into:
           [Diesel car, Petrol car, Hybrid/Electric/Eco car, Other car]

        Respond in this format strictly:

        Emotion: (write detected emotion in 1–2 words)
        Reasoning: (explain your thought in 8–15 words)
        Category: (final classification label)

        Text:
        {text}
        """

        try:
            response = model.generate_content(prompt)
            return (response.text or "").strip()

        except Exception as e:
            # Fallback still gives reasoning emotionally
            local_emotion = "Neutral"
            local_reason = "No strong emotion or car classifier keywords detected"
            local_cat = "Other car"
            for word in ["diesel","petrol","hybrid","electric","eco"]:
                if word in text.lower():
                    if word == "diesel":
                        local_cat = "Diesel car"
                    if word == "petrol":
                        local_cat = "Petrol car"
                    if word in ["hybrid","electric","eco"]:
                        local_cat = "Hybrid/Electric/Eco car"
            return f"Emotion: {local_emotion}\nReasoning: {local_reason}\nCategory: {local_cat}"


In [105]:
import os
import google.generativeai as genai
from google.adk.agents import Agent

# Configure API key (Gemini future-ready, Kaggle offline fallback)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

class ReasoningEmotionAgent(Agent):
    name: str = "reasoning_emotion_agent"
    description: str = "Classifies cars and text while providing emotion and reasoning."

    def run(self, text: str) -> str:
        prompt = f"""
        Analyze the emotion and classify the car or topic. Provide brief reasoning.

        Output format:
        Emotion: (one word)
        Reasoning: (one short line, max 12 words)
        Category: (final class)

        Text:
        {text}
        """
        try:
            response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
            return response.text.strip()
        except:
            # 🧠 Offline fallback reasoning and emotion using local rule logic
            t = text.lower()
            emotion = "Neutral"
            category = "Other car"

            if "diesel" in t:
                emotion = "Powerful"
                category = "Diesel car"
                reason = "Detected diesel keyword and strong premium vibe"
            elif "petrol" in t:
                emotion = "Concerned"
                category = "Petrol car"
                reason = "Petrol mentioned, possible cost concern"
            elif "hybrid" in t or "electric" in t or "eco" in t:
                emotion = "Hopeful"
                category = "Eco/Hybrid car"
                reason = "Sustainability terms detected"
            else:
                reason = "No fuel keyword matched, using 'Other'"

            return f"Emotion: {emotion}\nReasoning: {reason}\nCategory: {category}"


In [106]:
classifier = ReasoningEmotionAgent()  # ADK agent ✅


In [107]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Tata Nexon electric feels safe for my family and heart



🤖 Agent Response:

Emotion: Hopeful
Reasoning: Sustainability terms detected
Category: Eco/Hybrid car


In [108]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Toyota hybrid feels peaceful and environmentally kind



🤖 Agent Response:

Emotion: Hopeful
Reasoning: Sustainability terms detected
Category: Eco/Hybrid car


In [109]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Diesel sedan looks classy but fuel costs scare me



🤖 Agent Response:

Emotion: Powerful
Reasoning: Detected diesel keyword and strong premium vibe
Category: Diesel car


In [110]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Suggest me a car that feels safe and emotional with diesel option



🤖 Agent Response:

Emotion: Powerful
Reasoning: Detected diesel keyword and strong premium vibe
Category: Diesel car


In [111]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Hybrid cars make me feel responsible and smart



🤖 Agent Response:

Emotion: Hopeful
Reasoning: Sustainability terms detected
Category: Eco/Hybrid car


In [112]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  I feel hope when I see green technology on roads



🤖 Agent Response:

Emotion: Neutral
Reasoning: No fuel keyword matched, using 'Other'
Category: Other car


In [113]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  BMW 8 Series petrol but emotional fear of fuel shock



🤖 Agent Response:

Emotion: Concerned
Reasoning: Petrol mentioned, possible cost concern
Category: Petrol car


In [114]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Electric cars inspire hope but charging worries me



🤖 Agent Response:

Emotion: Hopeful
Reasoning: Sustainability terms detected
Category: Eco/Hybrid car


In [115]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  I love powerful automatic diesel engines



🤖 Agent Response:

Emotion: Powerful
Reasoning: Detected diesel keyword and strong premium vibe
Category: Diesel car


In [116]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  I love powerful automatic diesel engines



🤖 Agent Response:

Emotion: Powerful
Reasoning: Detected diesel keyword and strong premium vibe
Category: Diesel car


In [117]:
user_input = input("\nAsk something (text or car description): ")
output = classifier.run(user_input)   # ✅ Now gives emotion + reasoning + category

print("\n🤖 Agent Response:\n")
print(output)



Ask something (text or car description):  Petrol vehicles stress me but I still love them



🤖 Agent Response:

Emotion: Concerned
Reasoning: Petrol mentioned, possible cost concern
Category: Petrol car
